In [ ]:
#import the libraries

!pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#load the dataset
data_retail=pd.read_excel("Online retail.xlsx", sheet_name='Sheet1')


In [ ]:
data_retail

,"shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil"
0,"burgers,meatballs,eggs"
1,chutney
2,"turkey,avocado"
3,"mineral water,milk,energy bar,whole wheat rice..."
4,low fat yogurt
...,...
7495,"butter,light mayo,fresh bread"
7496,"burgers,frozen vegetables,eggs,french fries,ma..."
7497,chicken
7498,"escalope,green tea"


In [ ]:
# Split each transaction into a list of items and strip any leading/trailing whitespace
data_retail['items'] = data_retail.iloc[:, 0].str.split(',').apply(lambda x: [item.strip() for item in x])

# Ensure each item list contains unique items
data_retail['items'] = data_retail['items'].apply(lambda x: list(set(x)))

data_retail.head()

,"shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil",items
0,"burgers,meatballs,eggs","[eggs, burgers, meatballs]"
1,chutney,[chutney]
2,"turkey,avocado","[avocado, turkey]"
3,"mineral water,milk,energy bar,whole wheat rice...","[milk, whole wheat rice, energy bar, mineral w..."
4,low fat yogurt,[low fat yogurt]


In [ ]:
data_retail.shape

(7500, 2)

In [ ]:
data_retail.dtypes

shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil    object
items                                                                                                                                                                                                                              object
dtype: object

# Data Preprocessing

In [ ]:
data_retail.isnull().sum()

shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil    0
items                                                                                                                                                                                                                              0
dtype: int64

In [ ]:
# Create a list of all unique items
all_items = list(set(item for sublist in data_retail['items'] for item in sublist))

# One-hot encode the transactions
encoded_data = pd.DataFrame(0, index=data_retail.index, columns=all_items)

# unique is refered as 1 and 0 for non unique
for index, items in data_retail['items'].items():
    encoded_data.loc[index, items] = 1
encoded_data


#creates a DataFrame called encoded_data where each row represents a transaction and each column represents an item from the all_items list.
#Initially, all values are set to 0.


#All values in the DataFrame are initialized to 0.
#The idea is that each transaction will be one-hot encoded, where the presence of an item will be represented as 1, and absence as 0.

,ham,clothes accessories,asparagus,tomato juice,chocolate,cider,tea,whole wheat pasta,blueberries,melons,...,butter,shrimp,magazines,mayonnaise,soda,salmon,candy bars,frozen vegetables,fresh bread,body spray
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
encoded_data.shape    # 7500 rows and the 119 columns

(7500, 119)

In [ ]:
# Create a list of all unique items
unique_items = list(set(item for sublist in data_retail['items'] for item in sublist))
unique_items

['ham',
 'clothes accessories',
 'asparagus',
 'tomato juice',
 'chocolate',
 'cider',
 'tea',
 'whole wheat pasta',
 'blueberries',
 'melons',
 'french wine',
 'whole weat flour',
 'frozen smoothie',
 'mint',
 'tomatoes',
 'dessert wine',
 'sparkling water',
 'green grapes',
 'gluten free bar',
 'flax seed',
 'pancakes',
 'soup',
 'cereals',
 'bug spray',
 'babies food',
 'pet food',
 'chutney',
 'corn',
 'herb & pepper',
 'toothpaste',
 'oatmeal',
 'energy bar',
 'burger sauce',
 'water spray',
 'muffins',
 'pasta',
 'extra dark chocolate',
 'ketchup',
 'yams',
 'cookies',
 'chicken',
 'red wine',
 'yogurt cake',
 'spinach',
 'napkins',
 'carrots',
 'hand protein bar',
 'black tea',
 'cream',
 'parmesan cheese',
 'cottage cheese',
 'eggplant',
 'bramble',
 'meatballs',
 'white wine',
 'french fries',
 'chocolate bread',
 'ground beef',
 'cooking oil',
 'strong cheese',
 'light mayo',
 'low fat yogurt',
 'avocado',
 'mint green tea',
 'energy drink',
 'honey',
 'salad',
 'fresh tuna',

In [ ]:
# One-hot encode the transactions
encoded_data = pd.DataFrame(0, index=data_retail.index, columns=unique_items)
encoded_data.head()

,ham,clothes accessories,asparagus,tomato juice,chocolate,cider,tea,whole wheat pasta,blueberries,melons,...,butter,shrimp,magazines,mayonnaise,soda,salmon,candy bars,frozen vegetables,fresh bread,body spray
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# unique is refered as 1 and 0 for non unique
for index, items in data_retail['items'].items():
    encoded_data.loc[index, items] = 1
encoded_data

,ham,clothes accessories,asparagus,tomato juice,chocolate,cider,tea,whole wheat pasta,blueberries,melons,...,butter,shrimp,magazines,mayonnaise,soda,salmon,candy bars,frozen vegetables,fresh bread,body spray
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Association Rule Mining

In [ ]:
frequent_itemsets = apriori(encoded_data, min_support=0.01, use_colnames=True)

#min_support default=0.5
# min_support is a float between 0 and 1 for minumum support of the itemsets returned.
# The support is computed as the fraction transactions_where_item(s)_occur / total_transactions.

frequent_itemsets # output is frequent itemsets: 1-item itemsets, 2-item itemsets etc. with min support criteria

,support,itemsets
0,0.026533,(ham)
1,0.030267,(tomato juice)
2,0.163867,(chocolate)
3,0.010533,(cider)
4,0.029467,(whole wheat pasta)
...,...,...
254,0.014267,"(eggs, spaghetti, mineral water)"
255,0.015733,"(milk, spaghetti, mineral water)"
256,0.012000,"(spaghetti, frozen vegetables, mineral water)"
257,0.013067,"(eggs, milk, mineral water)"


In [ ]:
# Generate the association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.7) # min_threshold is Confidence
rules.sort_values('lift',ascending = False)[0:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
116,(herb & pepper),(ground beef),0.049467,0.098267,0.016000,0.323450,3.291555,0.011139,1.332841,0.732423
117,(ground beef),(herb & pepper),0.098267,0.049467,0.016000,0.162822,3.291555,0.011139,1.135402,0.772060
383,"(spaghetti, mineral water)",(ground beef),0.059733,0.098267,0.017067,0.285714,2.907540,0.011197,1.262427,0.697745
386,(ground beef),"(spaghetti, mineral water)",0.098267,0.059733,0.017067,0.173677,2.907540,0.011197,1.137893,0.727562
403,(olive oil),"(spaghetti, mineral water)",0.065733,0.059733,0.010267,0.156187,2.614731,0.006340,1.114306,0.661001
402,"(spaghetti, mineral water)",(olive oil),0.059733,0.065733,0.010267,0.171875,2.614731,0.006340,1.128171,0.656783
83,(tomatoes),(frozen vegetables),0.068400,0.095333,0.016133,0.235867,2.474134,0.009613,1.183913,0.639564
82,(frozen vegetables),(tomatoes),0.095333,0.068400,0.016133,0.169231,2.474134,0.009613,1.121370,0.658605
332,(shrimp),(frozen vegetables),0.071333,0.095333,0.016667,0.233645,2.450820,0.009866,1.180480,0.637444
333,(frozen vegetables),(shrimp),0.095333,0.071333,0.016667,0.174825,2.450820,0.009866,1.125418,0.654355


In [ ]:
rules[rules.lift>1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(frozen smoothie),(chocolate),0.063200,0.163867,0.014933,0.236287,1.441946,0.004577,1.094827,0.327170
1,(chocolate),(frozen smoothie),0.163867,0.063200,0.014933,0.091131,1.441946,0.004577,1.030732,0.366560
2,(tomatoes),(chocolate),0.068400,0.163867,0.014000,0.204678,1.249054,0.002792,1.051315,0.214034
3,(chocolate),(tomatoes),0.163867,0.068400,0.014000,0.085435,1.249054,0.002792,1.018627,0.238472
4,(pancakes),(chocolate),0.095067,0.163867,0.019867,0.208976,1.275282,0.004288,1.057027,0.238536
...,...,...,...,...,...,...,...,...,...,...
431,"(milk, mineral water)",(frozen vegetables),0.048000,0.095333,0.011067,0.230556,2.418415,0.006491,1.175740,0.616078
432,"(frozen vegetables, mineral water)",(milk),0.035733,0.129600,0.011067,0.309701,2.389672,0.006436,1.260904,0.603083
433,(milk),"(frozen vegetables, mineral water)",0.129600,0.035733,0.011067,0.085391,2.389672,0.006436,1.054294,0.668121
434,(frozen vegetables),"(milk, mineral water)",0.095333,0.048000,0.011067,0.116084,2.418415,0.006491,1.077025,0.648312


In [ ]:
rules= rules[(rules["confidence"] >= 0.5) & (rules["support"] >= 0.01) & (rules["lift"] >= 1.5)]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
388,"(eggs, ground beef)",(mineral water),0.020,0.238267,0.010133,0.506667,2.126469,0.005368,1.544054,0.540548
394,"(milk, ground beef)",(mineral water),0.022,0.238267,0.011067,0.503030,2.111207,0.005825,1.532756,0.538177


# Interpretation
There are 2 strong association given above after setting the approprate threshold, support and lift.

Rules with high lift values indicate that the presence of one item in a transaction increases the likelihood of another item being present.

Products that are frequently purchased together can be identified.

These insights can help in optimizing product placements, cross-selling, and promotional strategies.

# Interview Questions:

1.What is lift and why is it important in Association rules?

Lift is a metric used in association rule mining to evaluate the strength and significance of an association rule.Lift is a key metric in association rule mining because it provides a more nuanced view of the relationships between items by comparing the observed co-occurrence with the expected co-occurrence under independence.

2.What is support and Confidence. How do you calculate them?

Support measures how frequently an itemset appears in the dataset. It is the proportion of transactions that contain a particular itemset.confidence is a measure of the reliability or support for a given association rule. It is defined as the proportion of cases in which the association rule holds true.

3.What are some limitations or challenges of Association rules mining?

Association rule mining is a powerful technique but is challenged by issues such as computational complexity, large volumes of trivial or redundant rules, and difficulty in setting thresholds. Additionally, interpreting and acting on the results requires care, as the relationships identified are associations, not causalities.
